<a href="https://colab.research.google.com/github/PyotrAndreev/your-job-offer-1/blob/parser_and_matching/Match_user_and_vacancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2

In [4]:
!pip install rdflib

In [5]:
!pip install mpld3

In [6]:
!pip install sqlalchemy

In [12]:
!pip install FastAPI

In [13]:
!git clone https://github.com/Ksen1aT/your-job-offer-1.git

fatal: destination path 'your-job-offer-1' already exists and is not an empty directory.


In [14]:
%cd your-job-offer-1

/content/your-job-offer-1


In [15]:
!git switch data_base

M	main.py
Already on 'data_base'
Your branch is up to date with 'origin/data_base'.


In [16]:
!pip install Base

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import sys
sys.path.append('/content/your-job-offer-1/main.py')
sys.path.append('/content/your-job-offer-1/models.py')

In [17]:
!pip install sqlalchemy.orm

In [42]:
!pip install pydantic

In [1]:
!pip install typing

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

In [24]:
from main import read_vacancies

In [84]:
import sqlalchemy
import requests
from datetime import datetime, timedelta
import pandas as pd
import time
import random
import json
import time
import os
import json
import numpy as np
import re
from requests.exceptions import HTTPError, Timeout, ConnectionError
import rdflib
import nltk
from nltk.tokenize import sent_tokenize
from string import punctuation
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk import download as nltk_download
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import pymorphy2
from pymorphy2 import MorphAnalyzer

from sklearn import feature_extraction
import mpld3
from collections import Counter
import torch
from dataclasses import dataclass

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [11]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [86]:
from fastapi import FastAPI, HTTPException, Depends

In [92]:
from sqlalchemy import Column, Integer, BigInteger, String, Boolean, DateTime, ForeignKey, JSON, create_engine
from sqlalchemy.orm import relationship, sessionmaker, declarative_base
from datetime import datetime

# Создание базового класса для моделей
Base = declarative_base()

# Определение URL базы данных
DATABASE_URL = 'sqlite:///myDatabase.db'
engine = create_engine(DATABASE_URL)

class User(Base):
    __tablename__ = 'user'

    user_id = Column(BigInteger, primary_key=True)
    name = Column(JSON)
    email = Column(JSON)
    phone = Column(BigInteger)
    created_at = Column(DateTime, default=datetime.utcnow)  # Добавлено поле created_at

    resumes = relationship("Resume", back_populates="user")

    def __repr__(self):
        return f"<User(user_id={self.user_id}, name={self.name}, email={self.email}, phone={self.phone}, created_at={self.created_at})>"

class Company(Base):
    __tablename__ = 'company'

    company_id = Column(BigInteger, primary_key=True)
    company_title = Column(JSON)
    phone = Column(BigInteger)  # Контактный номер
    email = Column(JSON)  # Контактный email
    company_type = Column(JSON)
    description = Column(JSON)
    website = Column(JSON)
    created_at = Column(DateTime, default=datetime.utcnow)  # Добавлено поле created_at

    vacancies = relationship("Vacancy", back_populates="company")

    def __repr__(self):
        return f"<Company(company_id={self.company_id}, company_title={self.company_title}, phone={self.phone}, email={self.email}, created_at={self.created_at})>"

class Vacancy(Base):
    __tablename__ = 'vacancy'

    vacancy_id = Column(BigInteger, primary_key=True)
    company_id = Column(BigInteger, ForeignKey('company.company_id'))
    job_title = Column(JSON)
    response_letter_required = Column(Boolean)
    country = Column(JSON)
    city = Column(JSON)
    district = Column(JSON)
    salary = Column(BigInteger)
    office_address = Column(JSON)
    subway_station = Column(JSON)
    employer_information = Column(JSON)
    requirements = Column(JSON)
    work_schedule_working = Column(BigInteger)
    work_schedule_weekend = Column(BigInteger)
    experience = Column(JSON)
    remote_work = Column(Boolean)

    company = relationship("Company", back_populates="vacancies")
    submissions = relationship("Submission", back_populates="vacancy")

    def __repr__(self):
        return f"<Vacancy(vacancy_id={self.vacancy_id}, job_title={self.job_title}, salary={self.salary}, company_id={self.company_id}, created_at={self.created_at})>"

class Resume(Base):
    __tablename__ = 'resume'

    resume_id = Column(Integer, primary_key=True)
    user_id = Column(BigInteger, ForeignKey('user.user_id'))
    title = Column(JSON)
    job_title = Column(JSON)
    country = Column(JSON)
    city = Column(JSON)
    district = Column(JSON)
    min_salary = Column(BigInteger)
    max_salary = Column(BigInteger)
    work_schedule_working = Column(BigInteger)
    work_schedule_weekend = Column(BigInteger)
    experience = Column(JSON)
    remote_work = Column(Boolean)
    education = Column(JSON)
    additional_information = Column(JSON)


    user = relationship("User", back_populates="resumes")
    submissions = relationship("Submission", back_populates="resume")

    def __repr__(self):
        return f"<Resume(resume_id={self.resume_id}, title={self.title}, job_title={self.job_title}, user_id={self.user_id}, created_at={self.created_at})>"

class Submission(Base):
    __tablename__ = 'submission'
    submission_id = Column(BigInteger, primary_key=True)
    resume_id = Column(BigInteger, ForeignKey('resume.resume_id'))
    status = Column(JSON)
    sent_at = Column(DateTime)
    vacancy_id = Column(BigInteger, ForeignKey('vacancy.vacancy_id'))
    created_at = Column(DateTime, default=datetime.utcnow)  # Добавлено поле created_at

    resume = relationship("Resume", back_populates="submissions")
    vacancy = relationship("Vacancy", back_populates="submissions")

    def __repr__(self):
        return f"<Submission(submission_id={self.submission_id}, resume_id={self.resume_id}, vacancy_id={self.vacancy_id}, status={self.status}, created_at={self.created_at})>"

# Создание всех таблиц
Base.metadata.create_all(engine)


In [88]:
Session = sessionmaker(bind=engine)
session = Session()

In [61]:
def get_all_vacancies():
    vacancies = session.query(Vacancy).all()
    return vacancies

In [62]:
vacancies = get_all_vacancies()

In [63]:
@dataclass
class Vacancy_res:
    """Класс для хранения информации о вакансиях."""
    vacancy_id: int
    job_title: str
    response_letter_required: bool = False
    country: str = ""
    city: str = ""
    district: str = ""
    salary: list = None
    office_address: str = ""
    subway_station: str = ""
    employer_information: str = ""
    requirements: str = ""
    work_schedule_working_days: list = None
    work_schedule_time_intervals: list = None
    experience: str = ""
    remote_work: bool = False

In [73]:
def make_vacancy(vacancies):
  hh_vacancies = []
  for vacancy in vacancies:
    new_vacancy = Vacancy_res(vacancy_id = vacancy.vacancy_id,
                              job_title = str(vacancy.job_title),
                              response_letter_required = vacancy.response_letter_required,
                              country = str(vacancy.country),
                              city = str(vacancy.city),
                              district = str(vacancy.district),
                              office_address = str(vacancy.office_address),
                              subway_station = str(vacancy.subway_station),
                              employer_information = str(vacancy.employer_information),
                              requirements = str(vacancy.requirements),
                              experience = str(vacancy.experience),
                              remote_work = vacancy.remote_work)
    hh_vacancies.append(new_vacancy)
  return hh_vacancies


In [74]:
hh_vacancies = make_vacancy(vacancies)

In [94]:
def get_resume(resume_id = 1):
    resume = session.query(Resume).filter(Resume.resume_id == resume_id).first()
    return resume

In [95]:
resume_tmp = get_resume()

In [102]:
@dataclass
class Resume_res:
    resume_id: int
    user_id: int
    title: str = ""
    job_title: str = ""
    country: str = ""
    city: str = ""
    district: str = ""
    salary: int = 0
    work_schedule_working_days: list = None
    work_schedule_time_intervals: list = None
    experience: int = 0
    remote_work: bool = False
    education: str = ""
    additional_information: str = ""

In [103]:
def make_resume(resume):
    if resume == None:
      return
    new_resume = Resume_res(resume_id = resume.resume_id,
                            uer_id = resume.user_id,
                            title = str(resume.title),
                            job_title = str(resume.job_title),
                            country = str(resume.country),
                            district = str(resume.district),
                            salary = resume.min_salary,
                            experience = str(resume.experience),
                            additional_information = str(resume.additional_information),
                            education = str(resume.education),
                            remote_work = resume.remote_work)
    return new_resume

In [104]:
resume = make_resume(resume_tmp)

In [105]:
def clean(word):
    return re.sub(r"[^A-ZА-Яa-zа-я+\s]", "", word)

In [106]:
stop_words = stopwords.words('russian') + ["опыт",
                                           "навык",
                                           "знание",
                                           "понимание",
                                           "работа",
                                           "умение",
                                           "большой",
                                           "хороший",
                                           "прикладной",
                                           "некоторый",
                                           "небольшой"]


In [107]:
def tokenize_text(text):
    if text == None:
      return
    text = text.lower()
    text = text.translate(
        str.maketrans(punctuation, " " * len(punctuation))
    )
    word_tokens = word_tokenize(text)
    word_tokens = [clean(word) for word in word_tokens]
    morph = MorphAnalyzer()
    lemmatize_words = [morph.normal_forms(word)[0] for word in word_tokens]
    filtered_tokens = [word for word in lemmatize_words if word not in stop_words]
    return filtered_tokens

In [108]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

def get_word_embeddings(words):
    inputs = tokenizer(words, padding=True, truncation=True, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    embeddings = last_hidden_states[:, 0, :].cpu().numpy()
    return embeddings

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return float(dot_product / (norm_a * norm_b))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

In [109]:
def check_vacancy(requirements, skills):
  tok_req = tokenize_text(requirements)
  tok_skills = tokenize_text(skills)
  if tok_req == None or tok_skills == None:
    return False
  for i in tok_req:
    for j in tok_skills:
      if i == j:
        return True
  return False

In [110]:
def match_skills_and_requirements(requirements, skills):
  if requirements == None or skills == None:
    return 0
  emb_requirements = get_word_embeddings(requirements)
  emb_skills = get_word_embeddings(skills)
  similarity = 0
  max_similarity = 0
  for i in emb_requirements:
    for j in emb_skills:
      similarity += cosine_similarity(i, j)
      max_similarity += 1
  return similarity / max_similarity

In [111]:
currency = {'RUB': 1,
            'BYN': 31.05,
            'KZT': 0.19,
            'AMD': 0.25,
            'KGS': 1.16,
            'MDL': 5.5,
            'TJS': 9.22,
            'UZS': 0.0078}

In [113]:
def match_vacancies(vacancies, resume):
  similarity_with_vacancies = {}
  idx = 0
  for vacancy in vacancies:
    similarity = 0
    idx += 1
    if resume.country != "" and vacancy.country != "" and vacancy.country != resume.country:
      continue
    if resume.city != "" and vacancy.city != "" and vacancy.city != resume.city:
      continue
    if vacancy.remote_work == False and resume.remote_work == True:
      continue
    if check_vacancy(vacancy.requirements, resume.additional_information) == False:
      continue
    if vacancy.salary[0] != None and vacancy.salary[2] in currency:
       if vacancy.salary[1] != None and resume.salary * 0.9 <= vacancy.salary[1] * currency[vacancy.salary[2]]:
          similarity += 0.1
       elif vacancy.salary[1] == None and resume.salary * 1.1 >= vacancy.salary[0] * currency[vacancy.salary[2]]:
          similarity += 0.1
    experience_min = 0
    if vacancy.experience == 'От 1 года до 3 лет':
      experience_min = 1
    elif vacancy.experience == 'От 3 до 6 лет':
      experience_min = 3
    elif vacancy.experience == 'Более 6 лет':
      experience_min = 6
    if resume.experience >= experience_min:
      similarity += 0.1
    similarity += match_skills_and_requirements(vacancy.requirements, resume.additional_information)
    similarity += match_skills_and_requirements(vacancy.job_title, resume.job_title)
    if similarity in similarity_with_vacancies:
      similarity_with_vacancies[similarity].append(vacancy)
    else:
      similarity_with_vacancies[similarity] = [vacancy]
  return similarity_with_vacancies

In [ ]:
#resume = Resume_res(resume_id = 1,
#                user_id = 1,
#                additional_information = "автомобиль")

In [ ]:
#resume = Resume(resume_id = 1,
#                user_id = 1,
#                additional_information = "большой опыт в продажах")
#sim = match_vacancies(hh_vacancies[:200], resume)
#sim = sorted(sim.items(), reverse = True)
#num = 10
#print(sim[:num])

1
2
3
4
[(0.8875036835670471, [Vacancy(vacancy_id=112577058, job_title='Менеджер группы прямых продаж (DSA)', response_letter_required=False, country='Россия', city='', district='', salary=[60000, None, 'RUR'], office_address='', subway_station='', employer_information='Банк ВТБ (ПАО)', requirements='Среднее/среднее профессиональное, незаконченное/высшее образование. Опыт работы в продажах будет являться преимуществом. Желание развиваться в банковской сфере. ', work_schedule_working_days=None, work_schedule_time_intervals=None, experience='Нет опыта', remote_work=False)]), (0.8583747744560242, [Vacancy(vacancy_id=112622889, job_title='Менеджер по обучению и развитию', response_letter_required=False, country='Россия', city='Москва', district=None, salary=[280000, None, 'RUR'], office_address='Москва', subway_station='Таганская', employer_information='Alta Personnel', requirements='Высшее профессиональное образование (медицинское, фармацевтическое, психологическое или любое другое). Опыт

In [ ]:
#resume = Resume(resume_id = 1,
#                user_id = 1,
#                additional_information = "автомобили")
#sim = match_vacancies(hh_vacancies[:30], resume)
#sim = sorted(sim.items(), reverse = True)
#num = 10
#print(sim[:num])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
[(1.721099591255188, [Vacancy(vacancy_id=112593600, job_title='Водитель в семью', response_letter_required=False, country='Казахстан', city='', district='', salary=[300000, 400000, 'KZT'], office_address='', subway_station='', employer_information='Aipack', requirements='Имеющего опыт работы в семье. Аккуратное вождение. Порядочность. Пунктуальность. Наличие собственного автомобиля.', work_schedule_working_days=None, work_schedule_time_intervals=None, experience='Более 6 лет', remote_work=False)]), (1.7121224761009217, [Vacancy(vacancy_id=112595789, job_title='Водитель', response_letter_required=False, country='Казахстан', city='Астана', district='проспект Мангилик Ел', salary=[300000, 350000, 'KZT'], office_address='Астана, проспект Мангилик Ел, 20', subway_station='', employer_information='Мерей-Инжиниринг LTD', requirements='Личный автомобиль.', work_schedule_working_days=None, work_schedule_time_interv